In [1]:
def proj_omega(y,x_1,x_2):
    e_1 = x_1/np.linalg.norm(x_1)
    e_2 = x_2 - (x_2.T@e_1)*e_1
    e_2 = e_2/np.linalg.norm(e_2)
    
    y_proj_1 = (y.T@e_1)*e_1
    y_proj_2 = (y.T@e_2)*e_2
    
    return y_proj_1 + y_proj_2

In [2]:
def proj_vals(A,b,x_it,itr = 250,gamma = 1):
    
    error = np.zeros(itr)
    m = np.shape(A)[0]
    
    n = np.shape(X)
    
    proj = []
    
    for j in range(m): #Inner iteration for Kaczmarz updates
        a = A[j,:]
        proj_val = np.abs(b[j] - a@x_it)/np.linalg.norm(a)
        proj = proj + [proj_val]
    
    return proj


def high_proj(a, number = 5):

    res = sorted(range(len(a)), key = lambda sub: a[sub])[-number:]
    return res

def low_proj(a, number = 5):
    res = sorted(range(len(a)), key = lambda sub: a[sub])[:number]
    return res

In [3]:
def TIHT_HOSVD_proj_high(AA,yy,X,r,lamda = 1, itr = 100, num = 4): 
    
    n = np.shape(X)
    X_ravel = np.ravel(X)
    
    error = np.zeros(itr)
    
    vXX = torch.randn(n)*0

    for j in range(itr):
        
        proj = proj_vals(AA,yy,np.array(vectorize_tl(vXX)))
                           
        remove_row = high_proj(proj, number = num)
        A_1 = np.delete(AA,remove_row,axis = 0)
        y_1 = np.delete(yy,remove_row,axis = 0)
    
        WW = np.array(vectorize_tl(vXX)) + (m/(m-num))*lamda* np.matmul(A_1.T, (y_1 - np.matmul(A_1, np.array(vectorize_tl(vXX)))))
        WW = torch.reshape(torch.tensor(WW), n)
        vXX = HOSVD_rank_app(WW,r)
        error[j] = np.linalg.norm(vectorize_tl(vXX)- X_ravel)/np.linalg.norm(X_ravel)
          
    return vXX, error


def TIHT_HOSVD_proj_low(AA,yy,X,r,lamda = 1, itr = 100, num = 4): 
    
    n = np.shape(X)
    X_ravel = np.ravel(X)
    
    error = np.zeros(itr)
    
    vXX = torch.randn(n)*0

    for j in range(itr):
        
        proj = proj_vals(AA,yy,np.array(vectorize_tl(vXX)))
                           
        remove_row = low_proj(proj, number = num)
        A_1 = np.delete(AA,remove_row,axis = 0)
        y_1 = np.delete(yy,remove_row,axis = 0)
    
        WW = np.array(vectorize_tl(vXX)) + (m/(m-num))*lamda* np.matmul(A_1.T, (y_1 - np.matmul(A_1, np.array(vectorize_tl(vXX)))))
        WW = torch.reshape(torch.tensor(WW), n)
        vXX = HOSVD_rank_app(WW,r)
        error[j] = np.linalg.norm(vectorize_tl(vXX)- X_ravel)/np.linalg.norm(X_ravel)
          
    return vXX, error

In [7]:
def adaptive_TIHT(AA,yy,X,r,lamda,mu = 1,itr=250,numb=5,thresh = 1.1):
    
    n = np.shape(X)
    X_ravel = np.ravel(X)
    
    error = np.zeros(itr)
    error_clip = np.zeros(itr)
    delta_1_l = np.zeros(itr)
    rho_1_l = np.zeros(itr)
    rate_1_l = np.zeros(itr)
    delta_2_l = np.zeros(itr)
    rho_2_l = np.zeros(itr)
    rate_2_l = np.zeros(itr)
    
    vXX = torch.randn(n)*0
    vXX_unclip = torch.randn(n)*0

    for j in range(itr):
        
        proj = proj_vals(AA,yy,np.array(vectorize_tl(vXX)))
        
        #before clipping
        delta_1 = lamda*(np.linalg.norm(np.matmul(AA, vectorize_tl(vXX_unclip) - X_ravel))/np.linalg.norm(vectorize_tl(vXX_unclip) - X_ravel))**2
        WW_unclip = np.array(vectorize_tl(vXX_unclip)) + mu*lamda* np.matmul(AA.T, (yy - np.matmul(AA, np.array(vectorize_tl(vXX_unclip)))))
        WW_unclip = torch.reshape(torch.tensor(WW_unclip), n)
        R_t = vectorize_tl(vXX_unclip) - X_ravel
        vXX_unclip = HOSVD_rank_app(WW_unclip,r)
        R_t1 = vectorize_tl(vXX_unclip) - X_ravel
        rho_1 = (np.linalg.norm(proj_omega((1/np.sqrt(m)*AA.T)@(1/np.sqrt(m)*AA)@R_t,R_t,R_t1))/np.linalg.norm((1/np.sqrt(m)*AA)@R_t))**2
        rate_1 = 2*np.sqrt(1-(2-mu*rho_1)*mu*delta_1)
        delta_1_l[j] = delta_1
        rho_1_l[j] = rho_1
        rate_1_l[j] = rate_1
        error[j] = np.linalg.norm(vectorize_tl(vXX_unclip)- X_ravel)/np.linalg.norm(X_ravel)

        if delta_1 > thresh:
            num = numb
            remove_row = high_proj(proj, number = num)
            A_1 = np.delete(AA,remove_row,axis = 0)
            y_1 = np.delete(yy,remove_row,axis = 0)
        
        elif delta_1 < thresh:
            num = numb 
            remove_row = low_proj(proj, number = num)
            A_1 = np.delete(AA,remove_row,axis = 0)
            y_1 = np.delete(yy,remove_row,axis = 0)
    
        else:
            A_1 = AA
            num = 0
        
        #after clipping
        delta_2 = (m/(m-num))*lamda*(np.linalg.norm(np.matmul(A_1, vectorize_tl(vXX) - X_ravel))/np.linalg.norm(vectorize_tl(vXX) - X_ravel))**2
        WW = np.array(vectorize_tl(vXX)) + (m/(m-num))*lamda*mu*np.matmul(A_1.T, (y_1 - np.matmul(A_1, np.array(vectorize_tl(vXX)))))
        WW = torch.reshape(torch.tensor(WW), n)
        R_t = vectorize_tl(vXX) - X_ravel
        vXX = HOSVD_rank_app(WW,r)
        R_t1 = vectorize_tl(vXX) - X_ravel
        rho_2 = (m/(m-num))*lamda*(np.linalg.norm(proj_omega(A_1.T@A_1@R_t,R_t,R_t1))/np.linalg.norm(A_1@R_t))**2
        rate_2 =  2*np.sqrt(1-(2-mu*rho_2)*mu*delta_2)
        delta_2_l[j] = delta_2
        rho_2_l[j] = rho_2
        rate_2_l[j] = rate_2
        error_clip[j] = np.linalg.norm(vectorize_tl(vXX)- X_ravel)/np.linalg.norm(X_ravel)
    return delta_1_l, delta_2_l, rho_1_l, rho_2_l, rate_1_l, rate_2_l, error, error_clip
        
        